In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.chdir("/content/gdrive/MyDrive")
!ls

 2020fall-ml
'CBT test Diagram.drawio'
'CEN 5011 '
'Changhong Ma.pdf'
'Colab Notebooks'
'Copy of multi_participants_case_3 (acc).ipynb'
'CPT_Changhong Ma.pdf'
 CV.pdf
 Data_by_Participant
'Getting started.pdf'
 mind_reading.py
 mind_reading_v2.py
'New Panther Virtual Check In Module CERTIFICATE OF COMPLETION-Quiz Passed.pdf'
 oa
 __pycache__
 testing.ipynb
'UCD-User Account Home Page.drawio'
'Untitled Diagram.drawio'
'Untitled spreadsheet.gsheet'


In [ ]:
import mind_reading_v2 as mr
import pandas as pd
import re

In [ ]:
%cd "/content/gdrive/MyDrive/Data_by_Participant"

/content/gdrive/MyDrive/Data_by_Participant


In [ ]:
directory = os.fsencode("/content/gdrive/MyDrive/Data_by_Participant")
os.listdir(directory)

[b'001',
 b'007',
 b'004',
 b'003',
 b'006',
 b'010',
 b'012',
 b'011',
 b'009',
 b'016',
 b'020',
 b'023',
 b'017',
 b'013',
 b'021',
 b'019',
 b'015',
 b'014',
 b'018',
 b'033',
 b'026',
 b'027',
 b'029',
 b'031',
 b'030',
 b'025',
 b'034',
 b'024',
 b'032',
 b'044',
 b'038',
 b'040',
 b'043',
 b'041',
 b'036',
 b'039',
 b'042',
 b'037',
 b'035',
 b'048',
 b'049',
 b'055',
 b'053',
 b'054',
 b'046',
 b'047',
 b'050',
 b'051',
 b'052',
 b'cha',
 b'059',
 b'061',
 b'063',
 b'060',
 b'058',
 b'057',
 b'056',
 b'output',
 b'accuracy.csv']

In [ ]:
orig_participants = os.listdir(directory)
participants = []

for participant in orig_participants:
  # decode byte, make sure use the string type
  participant = participant.decode('utf-8')
  participants.append(participant)

# remove the 'cha' folder
participants.remove('cha')
participants

['001',
 '007',
 '004',
 '003',
 '006',
 '010',
 '012',
 '011',
 '009',
 '016',
 '020',
 '023',
 '017',
 '013',
 '021',
 '019',
 '015',
 '014',
 '018',
 '033',
 '026',
 '027',
 '029',
 '031',
 '030',
 '025',
 '034',
 '024',
 '032',
 '044',
 '038',
 '040',
 '043',
 '041',
 '036',
 '039',
 '042',
 '037',
 '035',
 '048',
 '049',
 '055',
 '053',
 '054',
 '046',
 '047',
 '050',
 '051',
 '052',
 '059',
 '061',
 '063',
 '060',
 '058',
 '057',
 '056',
 'output',
 'accuracy.csv']

In [ ]:
df = pd.DataFrame(index = ['SVC', 'DTC', 'NB', 'NN'])

def acc_df(df, acc_column, participant):
    '''
    Add accuracy for every participant to the whole results
    Args: 
        df: the dataframe of all results we have had
        acc_column: the dataframe of result we want to add 
        participant: participant number
    returns: 
        all accuracy results
    '''

    data = pd.DataFrame({f"Participant {participant}": acc_column})
    df[f"Participant {participant}"] = data[f"Participant {participant}"].values
    return df

In [ ]:
for participant in participants:
    # iterate all the folders

    for file in os.listdir(participant):
    # iterate all files in every folder, find out the one end with 'Cong.csv' and 'Incong.csv' as input data

        if file.endswith('Cong.csv'): file1 = f"{participant}/{file}" 
        if file.endswith('Incong.csv'): file2 = f"{participant}/{file}"

    # load in cong and incong data for them
    df1 = mr.load_data(file1)
    df2 = mr.load_data(file2)

    # concatenate such data 
    data = mr.concatenate_data(df1, df2)

    # find trials to later separate
    trials_index = mr.find_trials(data)

    # separate trials
    trials = mr.separate_trials(data, trials_index)

    # create the label column  
    labels = mr.create_multi_labels(data)

    # Go through each trial, reset the columns, we split from 100-300ms ((308th sample to 513th sample))
    pro_trials = mr.process_trials(trials)

    # Find the mean across channels
    avg_trials = mr.average_trials(pro_trials)

    # concatenates the average trials dataframe with labels
    ml_df = mr.create_ml_df(avg_trials, labels)

    # train models
    X_train, X_test, y_train, y_test = mr.prepare_ml_df(ml_df)

    acc_svc, precision_svc = mr.train_svc_multi(X_train, X_test, y_train, y_test)

    acc_dtc, precision_dtc = mr.train_dtc_multi(X_train, X_test, y_train, y_test)

    acc_nb, precision_nb  = mr.train_nb_multi(X_train, X_test, y_train, y_test)

    acc_nn, precision_nn = mr.train_nn_multi(64, X_train, X_test, y_train, y_test)

    # add every participant's accuracy together
    acc_list = [f"{acc_svc:.2f}", f"{acc_dtc:.2f}", f"{acc_nb:.2f}", f"{acc_nn:.2f}"]

    df = acc_df(df, acc_list, participant)    

        

    

Fitting 5 folds for each of 294 candidates, totalling 1470 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1470 out of 1470 | elapsed:   20.1s finished


Epoch 1/500
168/168 [==============================] - 1s 3ms/step - loss: 1.4169 - acc: 0.3015 - precision_m: 0.0446 - val_loss: 1.3908 - val_acc: 0.2569 - val_precision_m: 0.0000e+00
Epoch 2/500
168/168 [==============================] - 0s 2ms/step - loss: 1.4083 - acc: 0.2249 - precision_m: 0.0000e+00 - val_loss: 1.3888 - val_acc: 0.2431 - val_precision_m: 0.0000e+00
Epoch 3/500
168/168 [==============================] - 0s 2ms/step - loss: 1.3961 - acc: 0.2800 - precision_m: 0.0000e+00 - val_loss: 1.3913 - val_acc: 0.2014 - val_precision_m: 0.0000e+00
Epoch 4/500
168/168 [==============================] - 0s 2ms/step - loss: 1.3811 - acc: 0.2890 - precision_m: 0.0000e+00 - val_loss: 1.3978 - val_acc: 0.1875 - val_precision_m: 0.0000e+00
Epoch 5/500
168/168 [==============================] - 0s 2ms/step - loss: 1.3899 - acc: 0.2844 - precision_m: 0.0000e+00 - val_loss: 1.3939 - val_acc: 0.2500 - val_precision_m: 0.0000e+00
Epoch 6/500
168/168 [==============================] - 0s 2

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1470 out of 1470 | elapsed:   15.7s finished


Epoch 1/500
148/148 [==============================] - 1s 3ms/step - loss: 1.6580 - acc: 0.2462 - precision_m: 0.1317 - val_loss: 1.3834 - val_acc: 0.2441 - val_precision_m: 0.0000e+00
Epoch 2/500
148/148 [==============================] - 0s 2ms/step - loss: 1.4156 - acc: 0.2107 - precision_m: 0.0000e+00 - val_loss: 1.3814 - val_acc: 0.2756 - val_precision_m: 0.0000e+00
Epoch 3/500
148/148 [==============================] - 0s 2ms/step - loss: 1.3952 - acc: 0.2802 - precision_m: 0.0000e+00 - val_loss: 1.3786 - val_acc: 0.2913 - val_precision_m: 0.0000e+00
Epoch 4/500
148/148 [==============================] - 0s 2ms/step - loss: 1.3887 - acc: 0.2893 - precision_m: 0.0000e+00 - val_loss: 1.3815 - val_acc: 0.2441 - val_precision_m: 0.0000e+00
Epoch 5/500
148/148 [==============================] - 0s 2ms/step - loss: 1.3922 - acc: 0.2246 - precision_m: 0.0000e+00 - val_loss: 1.3809 - val_acc: 0.3071 - val_precision_m: 0.0000e+00
Epoch 6/500
148/148 [==============================] - 0s 2

...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1470 out of 1470 | elapsed:   19.6s finished


Epoch 1/500
152/152 [==============================] - 1s 3ms/step - loss: 1.4664 - acc: 0.2848 - precision_m: 0.0475 - val_loss: 1.3805 - val_acc: 0.2748 - val_precision_m: 0.0000e+00
Epoch 2/500
152/152 [==============================] - 0s 2ms/step - loss: 1.4043 - acc: 0.2420 - precision_m: 0.0000e+00 - val_loss: 1.3846 - val_acc: 0.2443 - val_precision_m: 0.0000e+00
Epoch 3/500
152/152 [==============================] - 0s 2ms/step - loss: 1.3890 - acc: 0.2748 - precision_m: 0.0000e+00 - val_loss: 1.3859 - val_acc: 0.2672 - val_precision_m: 0.0000e+00
Epoch 4/500
152/152 [==============================] - 0s 2ms/step - loss: 1.3858 - acc: 0.2531 - precision_m: 0.0000e+00 - val_loss: 1.3856 - val_acc: 0.2443 - val_precision_m: 0.0000e+00
Epoch 5/500
152/152 [==============================] - 0s 2ms/step - loss: 1.3868 - acc: 0.3032 - precision_m: 0.0000e+00 - val_loss: 1.3866 - val_acc: 0.2443 - val_precision_m: 0.0000e+00
Epoch 6/500
152/152 [==============================] - 0s 2

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1470 out of 1470 | elapsed:   19.2s finished


Epoch 1/500
152/152 [==============================] - 1s 4ms/step - loss: 1.4482 - acc: 0.2102 - precision_m: 0.0000e+00 - val_loss: 1.3825 - val_acc: 0.3231 - val_precision_m: 0.0000e+00
Epoch 2/500
152/152 [==============================] - 0s 2ms/step - loss: 1.3999 - acc: 0.1873 - precision_m: 0.0000e+00 - val_loss: 1.3890 - val_acc: 0.2385 - val_precision_m: 0.0000e+00
Epoch 3/500
152/152 [==============================] - 0s 2ms/step - loss: 1.3908 - acc: 0.2900 - precision_m: 0.0000e+00 - val_loss: 1.3894 - val_acc: 0.1923 - val_precision_m: 0.0000e+00
Epoch 4/500
152/152 [==============================] - 0s 2ms/step - loss: 1.3870 - acc: 0.2559 - precision_m: 0.0000e+00 - val_loss: 1.3840 - val_acc: 0.2923 - val_precision_m: 0.0000e+00
Epoch 5/500
152/152 [==============================] - 0s 2ms/step - loss: 1.3832 - acc: 0.2554 - precision_m: 0.0000e+00 - val_loss: 1.3897 - val_acc: 0.2231 - val_precision_m: 0.0000e+00
Epoch 6/500
152/152 [==============================] - 

NotADirectoryError: ignored

In [10]:
df

,Participant 001,Participant 007,Participant 004,Participant 003,Participant 006,Participant 010,Participant 012,Participant 011,Participant 009,Participant 016,Participant 020,Participant 023,Participant 017,Participant 013,Participant 021,Participant 019,Participant 015,Participant 014,Participant 018,Participant 033,Participant 026,Participant 027,Participant 029,Participant 031,Participant 030,Participant 025,Participant 034,Participant 024,Participant 032,Participant 044,Participant 038,Participant 040,Participant 043,Participant 041,Participant 036,Participant 039,Participant 042,Participant 037,Participant 035,Participant 048,Participant 049,Participant 055,Participant 053,Participant 054,Participant 046,Participant 047,Participant 050,Participant 051,Participant 052,Participant 059,Participant 061,Participant 063,Participant 060,Participant 058,Participant 057,Participant 056
SVC,0.33,0.27,0.40,0.35,0.39,0.26,0.36,0.41,0.39,0.37,0.30,0.55,0.30,0.31,0.29,0.27,0.27,0.27,0.36,0.24,0.31,0.25,0.25,0.33,0.30,0.28,0.31,0.31,0.27,0.25,0.31,0.32,0.30,0.23,0.28,0.35,0.32,0.26,0.27,0.26,0.23,0.21,0.37,0.32,0.28,0.26,0.28,0.21,0.28,0.25,0.25,0.25,0.26,0.23,0.24,0.25
DTC,0.25,0.24,0.33,0.27,0.24,0.28,0.34,0.41,0.19,0.25,0.25,0.52,0.24,0.23,0.30,0.24,0.25,0.27,0.21,0.24,0.22,0.28,0.25,0.41,0.23,0.24,0.20,0.25,0.23,0.21,0.35,0.36,0.25,0.28,0.25,0.15,0.22,0.23,0.23,0.36,0.28,0.31,0.34,0.32,0.31,0.19,0.28,0.21,0.26,0.24,0.26,0.29,0.26,0.22,0.23,0.22
NB,0.23,0.24,0.31,0.24,0.27,0.25,0.30,0.29,0.23,0.21,0.28,0.56,0.20,0.23,0.24,0.26,0.23,0.29,0.21,0.23,0.26,0.30,0.21,0.28,0.24,0.18,0.26,0.26,0.26,0.22,0.27,0.37,0.27,0.25,0.23,0.21,0.26,0.22,0.24,0.23,0.24,0.26,0.52,0.24,0.23,0.29,0.33,0.21,0.19,0.21,0.28,0.29,0.23,0.20,0.22,0.19
NN,0.20,0.29,0.36,0.34,0.29,0.26,0.26,0.23,0.23,0.20,0.25,0.60,0.27,0.23,0.24,0.24,0.25,0.29,0.26,0.29,0.23,0.19,0.28,0.24,0.22,0.21,0.29,0.24,0.21,0.25,0.27,0.46,0.24,0.23,0.28,0.17,0.28,0.25,0.29,0.28,0.27,0.24,0.31,0.27,0.26,0.15,0.34,0.18,0.28,0.27,0.28,0.26,0.29,0.25,0.24,0.27


In [12]:
df.to_csv('accuracy_multi_c4.csv') 